# 전처리
※ 주의사항: 전처리 관련 모든 파일은 현재 폴더에 저장해주세요

In [1]:
import pandas as pd
import numpy as np 
import re

In [2]:
pro_df = pd.read_csv('./크롤링 프로그램/programmers.csv', index_col=0)  # 기존
dart_df = pd.read_csv('./크롤링 프로그램/dart_result.csv', index_col=0)  # 다트
want_df = pd.read_csv('./크롤링 프로그램/wantedinsight_result.csv', index_col= 0)  # 원티드 인사이트
incru_df = pd.read_csv('./크롤링 프로그램/incruit_result.csv')  # 인크루트
cat_df = pd.read_csv('./크롤링 프로그램/catch_result.csv', index_col=0)  # 캐치

# 1. 프로그래머스 전처리
- 주소
- 사원수
- 직급

In [3]:
# 주소 전처리
address_list = []

for ad in pro_df['주소']:
    # 우편번호 패턴을 찾아서 제거
    ad = re.sub(r'^\d{5}\s', '', ad).strip()
    # 글자 변경
    if '대한민국' in ad:
        ad = re.sub('대한민국', '', ad).strip()
    elif '서울' in ad and '특별시' not in ad:
        ad = re.sub('서울','서울특별시',ad).strip()
    elif '경기' in ad and '경기도' not in ad:
        ad = re.sub('경기','경기도',ad).strip()
    elif '인천' in ad and '광역시' not in ad:
        ad = re.sub('인천','인천광역시',ad).strip()

    address_list.append(ad)

# 구,시,도로 끝나는 단어 리스트 정렬
add_list = [' '.join(re.findall(r'\b(\w+[구시도])\b', address)) for address in address_list]

pro_df['주소'] = add_list

In [4]:
# 사원수 전처리
employee_list = []

# '명' 및 공백 제거
for emp in pro_df['사원수']:
    employee_list.append(int(re.sub('명','',emp).strip()))

pro_df['사원수'] = employee_list
pro_df['사원수'] = pro_df['사원수'].astype(int)

In [5]:
# 직급 전처리
career_list = []

for car in pro_df['직급']:
    if '무관' not in car:  # 무관이 없는 경우 최소단위
        career_list.append(car.split('~')[0].strip())

    elif '신입' in car:  # 신입인 경우 0
        career_list.append(0)
                           
    else:  # 무관이 있는 경우 0
        career_list.append(0) 

pro_df['직급'] = career_list
pro_df['직급'] = pro_df['직급'].astype(int)  # int타입 변경

In [6]:
# 인덱스 재설정
pro_df.reset_index(drop=True, inplace=True)
# 전처리된 파일 저장
pro_df.to_csv('./pro_pre.csv')

# 2. 인크루트 전처리
- 설립연도
- 매출액
- 사원수
- 주소
- 기업 홈페이지 URL

In [7]:
# 설립연도
corp_year_df = []

for ss in incru_df.설립연도:
    if ss != '-':
        corp_year_df.append(int(''.join(re.findall(r'\d\d\d\d', ss))))
    else:
        corp_year_df.append(None)


incru_df.설립연도 = corp_year_df

In [8]:
# 매출액
sales_data = []
sales_df = []
data_sales_res = []

for sales in incru_df.매출액:
    if sales != '-':
        sales_data.append(sales.replace(',',''))
    else:
        sales_data.append(None)

for qq in sales_data:
    if qq != None:
        sales_df.append(re.sub(r'\(\d{4}\)', '', qq.replace(' ','')))
    else:
        sales_df.append(None)

for q in sales_df:
    if q != None: 
        if '조' in q:
                data_sales_res.append(int(''.join(re.findall(r'(\d+)조', q)))*100000000 + int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))
        elif '억원' in q:
                data_sales_res.append(int(''.join(re.findall(r'(\d+)억원', q)))*10000)
        elif '억' in q:
            data_sales_res.append(int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))   
        else:
            data_sales_res.append(int(''.join(re.findall(r'(\d+)만원', q))))
    else:
        data_sales_res.append(None)

incru_df.매출액 = data_sales_res

In [9]:
# 사원수
emp_df = []

for ww in incru_df.사원수:
    if ww != '-':
        emp_df.append(int(''.join(re.findall(r'\d', ww))))
    else:
        emp_df.append(None)

incru_df.사원수 = emp_df

In [10]:
# 주소
incru_df.주소.fillna(2)  # 결측치 채우기

address_list = []
add_list = []

for ee in incru_df.주소:
    try:    
        if '경기' in ee and '경기도' not in ee:
            ee = re.sub('경기', '경기도', ee)
        elif '서울' in ee and '서울특별시' not in ee:
             ee = re.sub('서울', '서울특별시', ee)
        elif '강남구' in ee and '서울특별시' not in ee and '서울' not in ee:
            ee = re.sub('강남구', '서울특별시 강남구', ee)
    except:
        ee = 0
        
    address_list.append(ee)

for tt in address_list:
    if tt != 0:
        add_list.append(' '.join(re.findall(r'\b(\w+[구시도])\b', tt)))
    else:
        add_list.append(None)

incru_df.주소 = add_list

In [11]:
# 기업 홈페이지 URL
incru_df['기업 홈페이지 URL'] = incru_df['기업 홈페이지 URL'].apply(lambda x: str(x)[2:])
incru_df.loc[incru_df['기업 홈페이지 URL'] == 'n', '기업 홈페이지 URL'] = None

In [12]:
# 인덱스 재설정
incru_df.reset_index(drop=True, inplace=True)
# 전처리된 파일 저장
incru_df.to_csv('./incru_pre.csv')

# 3. 다트 전처리
- 설립연도
- 주소

In [13]:
dart_df.설립연도 = dart_df.설립연도.apply(lambda x: ''.join(re.findall(r'\d\d\d\d', x)))  # 설립연도
dart_df.주소 = dart_df.주소.apply(lambda x: ' '.join(re.findall(r'\b(\w+[구시도])\b', x)))  # 주소
dart_df.설립연도 = dart_df.설립연도.astype(int)  # 타입변경

In [14]:
# 인덱스 재설정
dart_df.reset_index(drop=True, inplace=True)
# 전처리된 파일 저장
dart_df.to_csv('./dart_pre.csv')

# 4. 원티드 인사이트 전처리
- 업종분류
- 설립연도
- 매출액
- 사원수

In [15]:
# 잘못된 값 결측치로 변경
want_df.loc[want_df.업종분류 == '-', '업종분류'] = None  # 업종분류
want_df.loc[want_df.매출액 == '-원', '매출액'] = None  # 매출액

In [16]:
# 설립연도
want_df.설립연도 = want_df.설립연도.apply(lambda x: int(''.join(re.findall(r'(\d\d\d\d)', x))))

In [17]:
# 사원수 잘못된 값 결측치로 변경
want_df.loc[want_df.사원수 == '정보없음', '사원수'] = None
want_df.loc[want_df.사원수 == 'nan', '사원수'] = None

In [18]:
# 매출액
want_df_sales = []
want_df_sales_res = []

for q in want_df['매출액']:
    if q != None:
        want_df_sales.append(q.replace(',',''))
    else:
        want_df_sales.append(None)

try:
    for q in want_df_sales:
        if q != None: 
            if '조' in q:
                    want_df_sales_res.append(int(''.join(re.findall(r'(\d+)조', q)))*100000000 + int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))
            elif '억원' in q:
                 want_df_sales_res.append(int(''.join(re.findall(r'(\d+)억원', q)))*10000)
            elif '억' in q:
               want_df_sales_res.append(int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))   
            else:
                want_df_sales_res.append(int(''.join(re.findall(r'(\d+)만원', q))))
        else:
            want_df_sales_res.append(None)
except:
     print(q)

want_df['매출액'] = want_df_sales_res

In [19]:
# 사원수
want_df_emp = []

for f in want_df['사원수']:
    if pd.notna(f):
        want_df_emp.append(int(''.join(re.findall(r'\d', f))))
    else:
        want_df_emp.append(None)
        
want_df.사원수 = want_df_emp

In [20]:
# 인덱스 재설정
want_df.reset_index(drop=True, inplace=True)
# 전처리된 파일 저장
want_df.to_csv('./want_pre.csv')

# 5. 캐치 전처리
- 매출액
- 사원수
- 설립연도

In [21]:
# 컬럼명 변경: '설립일'을 '설립연도'로 변경
cat_df.rename(columns={'설립일': '설립연도', '상세업종': '업종분류', '기업규모': '기업형태'}, inplace=True)

In [22]:
# 매출액 전처리
m_list = []  
m_list2 = []  
m_list3 = []
# '조'와 '억'만 남기기
for m in cat_df['매출액']:
    if m == None:  # 결측치인 경우 str타입으로 변경
        m_list.append(str(m))
    elif '재무제표' in str(m):
        m_list.append(re.sub('\n', '', m.split(':')[-1].strip()))
    else:
        m_list.append(re.sub('\n', '', str(m)).split()[0])
# 억에서 , 제외하기
for m2 in m_list:
    m_list2.append(re.sub(',', '', m2))

# 조, 억, 만원을 만의단위로 변경하기
for m3 in m_list2:
    if m3 == '-' or m3 == 'nan':  # 결측치 처리
        m_list3.append(None)
    else:
        조_matches = re.findall(r'(\d+)조', m3)
        조 = int(조_matches[0]) * 10**8 if 조_matches else 0

        억_matches = re.findall(r'(\d+)억', m3)
        억 = int(억_matches[0]) * 10**4 if 억_matches else 0

        만원_matches = re.findall(r'(\d+)만원', m3)
        만원 = int(만원_matches[0]) if 만원_matches else 0

        m_list3.append(조 + 억 + 만원)
        
cat_df['매출액'] = m_list3

In [23]:
# 사원수
peo_list =[]

for p in cat_df['사원수']:
    try:
        if '-명' in p:
            peo_list.append(None)
        elif '명' in p:
            peo_list.append(int(re.findall(r'\d+', re.sub(',', '', p))[0]))
    except:
        peo_list.append(None)

cat_df['사원수'] = peo_list

In [24]:
# 설립연도
year_list = []

for y in cat_df['설립연도']:
    try:
        year_list.append(int(re.findall(r'\d+', y)[0]))
    except:
        year_list.append(None)
        
cat_df['설립연도'] = year_list

In [25]:
# 인덱스 재설정
cat_df.reset_index(drop=True, inplace=True)
# 전처리된 파일 저장
cat_df.to_csv('./cat_pre.csv')

# 기존 데이터에 전처리된 파일들 머지하기
- 머지 순서는 위에서 정리한 1번부터 5번까지 진행됩니다.

In [26]:
# 프로그래머스에 인크루트 머지
df_merge1 = pd.merge(pro_df, incru_df, on='기업명', how='left', suffixes=('', '_incru'))

# 합친 데이터프레임에서 인크루트에서 온 컬럼만 선택하기
cols_to_update = [col for col in df_merge1.columns if '_incru' in col]

# 원래 컬럼의 결측치를 인크루트에서 가져온 값으로 채우기
for col in cols_to_update:
    orig_col = col.replace('_incru', '')  # 원래 컬럼 이름
    # 원래 컬럼의 결측치를 채움
    df_merge1[orig_col] = df_merge1[orig_col].combine_first(df_merge1[col])

# 추가된 컬럼 제거
df_merge1.drop(columns=cols_to_update, inplace=True)

C:\Users\qhr22\AppData\Local\Temp\ipykernel_44148\795847017.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merge1[orig_col] = df_merge1[orig_col].combine_first(df_merge1[col])


In [27]:
# 다트 머지
df_merge2 = pd.merge(df_merge1, dart_df, on='기업명', how='left', suffixes=('', '_dart'))

# 합친 데이터프레임에서 다트에서 온 컬럼만 선택하기
cols_to_update = [col for col in df_merge2.columns if '_dart' in col]

# 원래 컬럼의 결측치를 다트에서 가져온 값으로 채우기
for col in cols_to_update:
    orig_col = col.replace('_dart', '')  # 원래 컬럼 이름
    # 원래 컬럼의 결측치를 채움
    df_merge2[orig_col] = df_merge2[orig_col].combine_first(df_merge2[col])

# 추가된 컬럼 제거
df_merge2.drop(columns=cols_to_update, inplace=True)

In [28]:
# 원티드 인사이트 머지
df_merge3 = pd.merge(df_merge2, want_df, on='기업명', how='left', suffixes=('', '_wanted'))

# 합친 데이터프레임에서 원티드 인사이트에서 온 컬럼만 선택하기
cols_to_update = [col for col in df_merge3.columns if '_wanted' in col]

# 원래 컬럼의 결측치를 원티드 인사이트에서 가져온 값으로 채우기
for col in cols_to_update:
    orig_col = col.replace('_wanted', '')  # 원래 컬럼 이름
    # 원래 컬럼의 결측치를 채움
    df_merge3[orig_col] = df_merge3[orig_col].combine_first(df_merge3[col])

# 추가된 컬럼 제거
df_merge3.drop(columns=cols_to_update, inplace=True)

C:\Users\qhr22\AppData\Local\Temp\ipykernel_44148\4098886825.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merge3[orig_col] = df_merge3[orig_col].combine_first(df_merge3[col])


In [29]:
# 캐치 머지
df_merge4 = pd.merge(df_merge3, cat_df, on='기업명', how='left', suffixes=('', '_catch'))

# 합친 데이터프레임에서 캐치에서 온 컬럼만 선택하기
cols_to_update = [col for col in df_merge4.columns if '_catch' in col]

# 원래 컬럼의 결측치를 캐치에서 가져온 값으로 채우기
for col in cols_to_update:
    orig_col = col.replace('_catch', '')  # 원래 컬럼 이름
    # 원래 컬럼의 결측치를 채움
    df_merge4[orig_col] = df_merge4[orig_col].combine_first(df_merge4[col])

# 추가된 컬럼 제거
df_merge4.drop(columns=cols_to_update, inplace=True)

C:\Users\qhr22\AppData\Local\Temp\ipykernel_44148\2481769147.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merge4[orig_col] = df_merge4[orig_col].combine_first(df_merge4[col])


In [30]:
# 중복된 값 제거 및 index 재설정
df_merge4 = df_merge4.drop_duplicates().reset_index(drop=True)
# 컬럼 순서 지정
df_merge4 = df_merge4[['기업명', '업종분류', '사원수', '설립연도', '기업형태', '매출액', '주소', '직무', '이용하는 기술스택',
            '자격요건', '직급', '우대사항', '해당 페이지 URL', '기업 홈페이지 URL']]

# 최종 파일 저장
df_merge4.to_csv('./programmers_end.csv')
df_merge4.to_excel('./programmers_end.xlsx')